In [4]:
import numpy as np
import pandas as pd

### 辅助函数

https://blog.whuzfb.cn/blog/2020/08/19/time_system/

In [1088]:
import time
from datetime import datetime, timedelta

# 闰秒
LEAP_SECONDS = 18

"""
【功能】GPS => UTC
【输入】GPS周、GPS周内秒、闰秒（可选，gps时间不同，闰秒值也不同，由Leap_Second.dat文件决定）
【输出】UTC时间（格林尼治时间）

【输入示例】gps_week_seconds_to_utc(2119, 214365.000)
【输出示例】'2020-08-18 11:32:27.000000'
"""
def gps_week_seconds_to_utc(gpsweek, gpsseconds, leapseconds=LEAP_SECONDS):
    datetimeformat = "%Y-%m-%d %H:%M:%S.%f"
    epoch = datetime.strptime("1980-01-06 00:00:00.000", datetimeformat)
    # timedelta函数会处理seconds为负数的情况
    elapsed = timedelta(days=(gpsweek*7), seconds=(gpsseconds-leapseconds))
    return datetime.strftime(epoch+elapsed, datetimeformat)

"""
【功能】UTC => GPS
【输入】UTC时间（datetime类型）
【输出】GPS周、周内日、周内秒、毫秒
"""
def utc_to_gps_week_seconds(utc, leapseconds=LEAP_SECONDS):
    datetimeformat = "%Y-%m-%d %H:%M:%S.%f"
    epoch = datetime.strptime("1980-01-06 00:00:00.000", datetimeformat)
    tdiff = utc - epoch + timedelta(seconds=leapseconds)
    gpsweek = tdiff.days // 7
    gpsdays = tdiff.days - 7*gpsweek
    gpsseconds = tdiff.seconds + 86400*(tdiff.days -7*gpsweek)
    return gpsweek, gpsdays, gpsseconds, tdiff.microseconds

"""
[TEST]
"""
now_utc = datetime.utcnow() # 获取当前时间UTC
print("<UTC>", now_utc)
print("<UTC Format>", now_utc.strftime('%Y-%m-%d %H:%M:%S.%f'))
print("<DateTime String>", datetime.strptime("2023-08-08 07:21:15.300000", '%Y-%m-%d %H:%M:%S.%f'))
print("<GPST>", utc_to_gps_week_seconds(now_utc))
print(utc_to_gps_week_seconds(datetime.strptime("2023-08-08 07:21:15.700000", '%Y-%m-%d %H:%M:%S.%f')))
print(utc_to_gps_week_seconds(datetime.strptime("2023-08-08 09:39:50.900000", '%Y-%m-%d %H:%M:%S.%f')))

<UTC> 2023-09-22 02:48:02.054480
<UTC Format> 2023-09-22 02:48:02.054480
<DateTime String> 2023-08-08 07:21:15.300000
<GPST> (2280, 5, 442100, 54480)
(2274, 2, 199293, 700000)
(2274, 2, 207608, 900000)


### 读取数据

#### 1.输入数据一（stim300_Inertial_data_m.txt）

In [1089]:
pd.set_option('display.float_format',lambda x : '%.6f' % x)
x_stim300_inertial = pd.read_csv(
    "data/stim300_Inertial_data_m.txt", 
    sep = " ", 
    header = None, names = ["ts_stim300_inertial", "S1", "S2", "S3", "S4", "S5", "S6"])
print("<stim300_Inertial_data_m shape>: ", x_stim300_inertial.shape)
x_stim300_inertial.head(10)

<stim300_Inertial_data_m shape>:  (1040092, 7)


,ts_stim300_inertial,S1,S2,S3,S4,S5,S6
0,199275.300000,-0.002584,-0.000145,-0.000162,-0.010966,0.150204,-9.716401
1,199275.308000,-0.002486,-0.002648,0.001924,0.025744,0.090851,-9.726078
2,199275.316000,-0.000730,-0.000652,-0.000679,-0.040970,0.158125,-9.747431
3,199275.324000,0.001160,-0.000761,-0.001168,-0.008370,0.142750,-9.697154
4,199275.332000,0.000614,0.001307,0.000641,0.043922,0.141198,-9.678172
5,199275.340000,-0.001513,-0.000876,0.000428,-0.044818,0.177535,-9.749410
6,199275.348000,-0.000879,-0.000158,0.002394,0.029499,0.133166,-9.756560
7,199275.356000,0.002297,-0.001174,0.001311,-0.046761,0.188931,-9.692394
8,199275.364000,-0.001418,0.001521,0.000764,-0.028397,0.156574,-9.746794
9,199275.372000,0.000525,-0.001393,0.001285,-0.012050,0.135445,-9.702199


#### 2.输入数据集二（bdStar_front.23O）

In [1091]:
"""
【步骤一】读取非结构化文本数据
"""
x_bdstar_front = pd.read_csv("data/bdStar_front.23O", header = None, names = ["text"])
x_bdstar_front.head(5)

,text
0,3.02 OBSERVATION DATA M (MIX...
1,Convert 2.6.6 NovAtel 202308...
2,RINEX Version 3.02 Observation File ...
3,...
4,...


In [1092]:
"""
【步骤二】将非结构化文本数据转换为表格数据
"""
# 构建数据块索引
# 寻找数据域边界
data_block_tag = x_bdstar_front[x_bdstar_front.text.str.find("END OF HEADER") != -1]
data_block_tag_idx = data_block_tag.index.values[0]
data_block = x_bdstar_front.iloc[data_block_tag_idx + 1:]

# 寻找每个时间区间数据边界生成索引
chunck_index = {}
data_chunck_tag = data_block[data_block.text.str.startswith(">")]
data_chunck_tag_array = data_chunck_tag.index.values
data_chunck_tag_array_size = data_chunck_tag_array.size
for i in range(data_chunck_tag_array_size): 
    start = data_chunck_tag_array[i]
    end = -1 if i == data_chunck_tag_array_size -1 else data_chunck_tag_array[i + 1]
    k = (start, end)
    v = x_bdstar_front.iloc[start]["text"].replace("> ", "")
    chunck_index[k] = v

In [1093]:
import os
import time

# 定义行文本解析函数
def __raw_split(raw):
    # raw: pandas.core.series.Series
    _ts  = raw.loc["_ts"]
    _text = raw.loc["text"]
    # 将字符串转换为字符数组
    chs = list(_text)
    # 按照固有字符站位取值
    '''
    _text: G18  20978312.758 6 110241789.652 6      1315.434          40.000 ......
    
    _value             _offset 
    G18                0  3
    20978312.758       3  17
    6                  17 19
    110241789.652      19 33
    6                  34 35
    1315.434           35 49
    40.000             49 65
    '''
    _No = "".join(chs[0: 3]).strip()
    _B1 = "".join(chs[3: 17]).strip()
    _B2 = "".join(chs[17: 19]).strip()
    _B3 = "".join(chs[19: 33]).strip()
    _B4 = "".join(chs[34: 35]).strip()
    _B5 = "".join(chs[35: 49]).strip()
    _B6 = "".join(chs[49: 65]).strip()
    return pd.Series([_ts, _No, _B1, _B2, _B3, _B4, _B5, _B6], index = ["_ts", "No", "B1", "B2", "B3", "B4", "B5", "B6"])


dataset_path = "result/bd_star_front.csv"
if os.path.exists(dataset_path):
    os.remove(dataset_path)

_start = time.time()

# 将非结构化文本数据转换为表格数据并分块存储
for k, v in chunck_index.items():
    '''
    [DEBUG] 
    # k = (40, 63)
    # v = chunck_index[k]
    '''
    start = k[0] + 1
    end = k[1]
    # 获取数据块
    chunck = x_bdstar_front.iloc[start:] if end == -1 else x_bdstar_front.iloc[start: end]
    # 增加时间戳列
    chunck.insert(0, "_ts", v)
    # 解析文本中的有效数据列
    chunck = chunck.apply(__raw_split, axis = 1)
    # 分块追加保存到文件
    chunck.to_csv(dataset_path, index = None, header = None, sep = ',', mode = 'a')

_end = time.time()
print("rewrite <bdStar_front.23O> for time(s): ", _end - _start)

rewrite <bdStar_front.23O> for time(s):  186.97577905654907


In [1094]:
"""
【步骤三】查看处理为表格化的数据集
"""
pd.set_option('display.float_format',lambda x : '%.3f' % x)

dataset_path = "result/bd_star_front.csv"
x_bd_star = pd.read_csv(
    dataset_path,
    sep = ",", 
    header = None,
    names = ["_ts", "No", "B1", "B2", "B3", "B4", "B5", "B6"])

print("<x_bd_star shape>: ", x_bd_star.shape)
x_bd_star.head(5)

<x_bd_star shape>:  (1768456, 8)


,_ts,No,B1,B2,B3,B4,B5,B6
0,2023 08 08 07 21 15.3000000 0 22 -0.0000...,G18,20978312.758,6.000,110241789.652,6.000,1315.434,40.000
1,2023 08 08 07 21 15.3000000 0 22 -0.0000...,G15,20304953.867,8.000,106703256.668,8.000,-1119.289,51.000
2,2023 08 08 07 21 15.3000000 0 22 -0.0000...,G20,24364894.555,7.000,128038351.324,7.000,-2696.160,47.000
3,2023 08 08 07 21 15.3000000 0 22 -0.0000...,G05,22477802.289,8.000,118121627.969,8.000,-1981.223,49.000
4,2023 08 08 07 21 15.3000000 0 22 -0.0000...,G24,20799438.789,8.000,109301798.004,8.000,2271.996,51.000


#### ~3.输出数据集一（n580-2023-08-08 05-38-20_.csv）~

该数据集可以理解为机器学习中的Y值（多个）

In [1095]:
pd.set_option('display.float_format',lambda x : '%.10f' % x)

y_n580 = pd.read_csv(
    "data/n580-2023-08-08 05-38-20_.csv", 
    sep = ",")

y_n580.columns = ["Message", "SystemTOV", "GPSTOV", "TimeSinceLastMsg", "LinearAccelerationX", "LinearAccelerationY", "LinearAccelerationZ", "AngularRateX", "AngularRateY", "AngularRateZ"]

print("<n580-2023-08-08 05-38-20 shape>: ", y_n580.shape)

y_n580.head(5)

<n580-2023-08-08 05-38-20 shape>:  (1236652, 10)


,Message,SystemTOV,GPSTOV,TimeSinceLastMsg,LinearAccelerationX,LinearAccelerationY,LinearAccelerationZ,AngularRateX,AngularRateY,AngularRateZ
0,0x2311,2115.5273374781,199291.3207896800,0.0000000000,-9.8040680000,-0.0266112400,-0.0574022500,0.0002311426,-0.0006141607,-0.0004310627
1,0x2311,2115.5273374781,199291.3207896800,0.0000000000,-9.8040680000,-0.0266112400,-0.0574022500,0.0002311426,-0.0006141607,-0.0004310627
2,0x2311,2115.5273374781,199291.3207896800,0.0000000000,-9.8040680000,-0.0266112400,-0.0574022500,0.0002311426,-0.0006141607,-0.0004310627
3,0x2311,2115.5273374781,199291.3207896800,0.0000000000,-9.8040680000,-0.0266112400,-0.0574022500,0.0002311426,-0.0006141607,-0.0004310627
4,0x2311,2115.5373374781,199291.3307896800,0.0100000000,-9.8241590000,-0.0149687800,-0.0490877800,0.0001754728,0.0000233646,-0.0001641456


#### 3.输出数据集一（refPath_frontAnt）

该数据集可以理解为机器学习中的Y值（多个）

In [1096]:
ref_path_front_ant_path = "data/refPath_frontAnt"

# 读取不规则文件
# 参数on_bad_lines='skip'表示：跳过损坏的行不抛出异常 
broken_data_block = pd.read_csv(ref_path_front_ant_path, header = None, names = ["text"], on_bad_lines = 'skip')

# 去除非结构化的行
data_block = broken_data_block[(broken_data_block.text.str.find(":") == -1) & (broken_data_block.text.str.find("(") == -1)]

# 获取文件中的表头
y_n580_columns = data_block.iloc[0].tolist()[0].strip().split()

# 去除数据集中的表头
data_block = data_block.drop(10)

# 定义行解析函数
def __row(r):
    s = r.tolist()[0]
    return pd.Series(s.split(), index = y_n580_columns)
    
# 将文本列转换为多列
y_n580 = data_block.apply(__row, axis = 1).astype('float64')

print("<refPath_frontAnt>: ", y_n580.shape)
y_n580.head(5)

<refPath_frontAnt>:  (82261, 11)


,Week,GPSTime,Latitude,Longitude,H-Ell,VEast,VNorth,VUp,Roll,Pitch,Heading
12,2274.0000000000,199292.0000000000,31.9395290903,118.7859434824,21.4460000000,0.0000000000,-0.0020000000,0.0020000000,-0.0410638852,-0.3712709217,156.9934897309
13,2274.0000000000,199292.1000000000,31.9395290905,118.7859434823,21.4460000000,0.0000000000,-0.0010000000,0.0020000000,-0.0412990819,-0.3700717945,156.9935477645
14,2274.0000000000,199292.2000000000,31.9395290907,118.7859434824,21.4460000000,0.0000000000,-0.0020000000,0.0020000000,-0.0409016418,-0.3695781988,156.9929744085
15,2274.0000000000,199292.3000000000,31.9395290904,118.7859434823,21.4460000000,0.0000000000,-0.0020000000,0.0020000000,-0.0406150487,-0.3707854973,156.9933462074
16,2274.0000000000,199292.4000000000,31.9395290904,118.7859434822,21.4460000000,-0.0000000000,-0.0010000000,0.0020000000,-0.0408315255,-0.3707798790,156.9937511923


### 数据清洗

#### 1.数据集一

无需清洗

In [1097]:
train_stim300 = x_stim300_inertial
train_stim300["millis_stim300_inertial"] = train_stim300["ts_stim300_inertial"].map(_1f)
print("<stim300>: ", train_stim300.shape)
train_stim300.head(5)

<stim300>:  (1040092, 8)


,ts_stim300_inertial,S1,S2,S3,S4,S5,S6,millis_stim300_inertial
0,199275.3000000000,-0.0025840000,-0.0001450000,-0.0001620000,-0.0109660000,0.1502040000,-9.7164010000,199275.3000000000
1,199275.3080000000,-0.0024860000,-0.0026480000,0.0019240000,0.0257440000,0.0908510000,-9.7260780000,199275.3000000000
2,199275.3160000000,-0.0007300000,-0.0006520000,-0.0006790000,-0.0409700000,0.1581250000,-9.7474310000,199275.3000000000
3,199275.3240000000,0.0011600000,-0.0007610000,-0.0011680000,-0.0083700000,0.1427500000,-9.6971540000,199275.3000000000
4,199275.3320000000,0.0006140000,0.0013070000,0.0006410000,0.0439220000,0.1411980000,-9.6781720000,199275.3000000000


#### 2.数据集二

In [1102]:
# 定义时间列解析函数

def __time_format(ts):
    _str = ts.split()
    Y = _str[0]
    m = _str[1]
    d = _str[2]
    H = _str[3]
    M = _str[4]
    S = _str[5].split(".")[0]
    f = int(float(_str[5].split(".")[1]) / 10)
    return f"{Y}-{m}-{d} {H}:{M}:{S}.{f}"

def __time_to_gpst(ts):
    _str = ts.split()
    Y = _str[0]
    m = _str[1]
    d = _str[2]
    H = _str[3]
    M = _str[4]
    S = _str[5].split(".")[0]
    f = int(float(_str[5].split(".")[1]) / 10)
    _data = f"{Y}-{m}-{d} {H}:{M}:{S}.{f}"
    (gps_week, gps_days, gps_seconds, gps_seconds_microseconds) = \
        utc_to_gps_week_seconds(datetime.strptime(_data, '%Y-%m-%d %H:%M:%S.%f'))
    return float(gps_seconds) + float(format(gps_seconds_microseconds / 1000000.0, ".6f"))

"""
[TEST]
"""
print(__time_format("2023 08 08 07 21 15.4000000  0 22      -0.000000000000"))
print(__time_to_gpst("2023 08 08 07 21 15.4000000  0 22      -0.000000000000"))

2023-08-08 07:21:15.400000
199293.4


In [1103]:
# ##########################
# 数据透视（对数据集进行列转行）
# ##########################

# 数据透视转换（两级列索引）
# 将相同日期的数据打平成一行，设置透视图的行索引（index）
index_cols = "_ts"
# 根据相同设备号的数据组织透视图，设置透视图的列索引（columns）
cols = "No"
# 设置透视指标列（values）
pivot_cols = ["B1", "B2", "B3", "B4", "B5", "B6"]
# 生成透视图
pivot_tb = x_bd_star.pivot(index = index_cols, columns = cols, values = pivot_cols)

# ##########################
# 多层级列索引处理
# ##########################

# 交换内外索引
pivot_tb = pivot_tb.swaplevel(axis = 1)
# 多级列索引排序
sorted_column_index = pivot_tb.columns.sort_values()
# 重制多级列索引
pivot_tb = pivot_tb.reindex(sorted_column_index, axis = 1)
# 多级列索引压缩为一级列索引（使用下划线连接）
pivot_tb.columns = pivot_tb.columns.map('_'.join) 
# 重制单级列索引，并将数据透视图的行索引转换为一列
pivot_tb = pivot_tb.reset_index()
"""
[DEBUG]
"""
# pivot_tb.head(20)

# train_bd_star = pivot_tb.dropna(axis = 0)
train_bd_star = pivot_tb
train_bd_star["ts"] = train_bd_star["_ts"].map(__time_format)
train_bd_star["gpst"] = train_bd_star["_ts"].map(__time_to_gpst)
# 在原有数据集中删除_ts这一列
train_bd_star.drop("_ts", axis = 1, inplace = True)
train_bd_star["millis_bd_star"] = train_bd_star["gpst"].map(_1f)
print("<train_bd_star>: ", train_bd_star.shape)
train_bd_star.head(5)

<train_bd_star>:  (82871, 267)


,C01_B1,C01_B2,C01_B3,C01_B4,C01_B5,C01_B6,C02_B1,C02_B2,C02_B3,C02_B4,...,R23_B6,R24_B1,R24_B2,R24_B3,R24_B4,R24_B5,R24_B6,ts,gpst,millis_bd_star
0,37125346.7030000016,7.0000000000,193321423.8400000036,7.0000000000,-31.1130000000,47.0000000000,NaN,NaN,NaN,NaN,...,29.0000000000,20203824.1640000008,6.0000000000,108038547.1889999956,6.0000000000,121.0040000000,38.0000000000,2023-08-08 07:21:15.300000,199293.3000000000,199293.3000000000
1,37125347.2969999984,7.0000000000,193321426.9339999855,7.0000000000,-30.8980000000,47.0000000000,NaN,NaN,NaN,NaN,...,29.0000000000,20203821.9140000008,6.0000000000,108038535.1060000062,6.0000000000,120.8320000000,38.0000000000,2023-08-08 07:21:15.400000,199293.4000000000,199293.4000000000
2,37125347.9060000032,7.0000000000,193321430.0740000010,7.0000000000,-31.3590000000,47.0000000000,NaN,NaN,NaN,NaN,...,29.0000000000,20203819.6559999995,6.0000000000,108038523.0359999985,6.0000000000,120.5230000000,38.0000000000,2023-08-08 07:21:15.500000,199293.5000000000,199293.5000000000
3,37125348.5080000013,7.0000000000,193321433.2030000091,7.0000000000,-31.3710000000,47.0000000000,NaN,NaN,NaN,NaN,...,29.0000000000,20203817.4140000008,6.0000000000,108038511.0279999971,6.0000000000,120.2620000000,38.0000000000,2023-08-08 07:21:15.600000,199293.6000000000,199293.6000000000
4,37125349.1019999981,7.0000000000,193321436.3199999928,7.0000000000,-31.1450000000,47.0000000000,NaN,NaN,NaN,NaN,...,29.0000000000,20203815.1640000008,6.0000000000,108038498.9930000007,6.0000000000,120.3130000000,38.0000000000,2023-08-08 07:21:15.700000,199293.7000000000,199293.7000000000


#### 3.数据集三

无需清洗

In [1105]:
pd.set_option('display.float_format',lambda x : '%.10f' % x)

# 删除缺失值的行
train_n580 = y_n580.dropna(axis = 0)

# train_n580["millis_n580"] = train_n580["GPSTOV"].map(_1f)
train_n580["millis_n580"] = train_n580["GPSTime"].map(_1f)

print("<train_n580>: ", train_n580.shape)
train_n580.head(5)

<train_n580>:  (82261, 12)


,Week,GPSTime,Latitude,Longitude,H-Ell,VEast,VNorth,VUp,Roll,Pitch,Heading,millis_n580
12,2274.0000000000,199292.0000000000,31.9395290903,118.7859434824,21.4460000000,0.0000000000,-0.0020000000,0.0020000000,-0.0410638852,-0.3712709217,156.9934897309,199292.0000000000
13,2274.0000000000,199292.1000000000,31.9395290905,118.7859434823,21.4460000000,0.0000000000,-0.0010000000,0.0020000000,-0.0412990819,-0.3700717945,156.9935477645,199292.1000000000
14,2274.0000000000,199292.2000000000,31.9395290907,118.7859434824,21.4460000000,0.0000000000,-0.0020000000,0.0020000000,-0.0409016418,-0.3695781988,156.9929744085,199292.2000000000
15,2274.0000000000,199292.3000000000,31.9395290904,118.7859434823,21.4460000000,0.0000000000,-0.0020000000,0.0020000000,-0.0406150487,-0.3707854973,156.9933462074,199292.3000000000
16,2274.0000000000,199292.4000000000,31.9395290904,118.7859434822,21.4460000000,-0.0000000000,-0.0010000000,0.0020000000,-0.0408315255,-0.3707798790,156.9937511923,199292.4000000000


### 数据规整

In [1106]:
"""
【步骤一】   train_bd_star  <left join> train_stim300
【关联键】   millis_bd_star             millis_stim300_inertial
【关联条件】 关联后的gpst和ts_stim300_inertial差尽可能的小，最好是0，如果不是0取差值最小的
"""
dataset = train_bd_star.merge(train_stim300, 
                              left_on = "millis_bd_star", 
                              right_on = "millis_stim300_inertial", 
                              how = 'left').drop_duplicates()

dataset["gap"] = abs(dataset["gpst"] - dataset["ts_stim300_inertial"])

def _min_gap(x):
    df = x.sort_values(by = 'gap', ascending = False)
    return df.iloc[-1,:]
    
dataset = dataset.groupby(by = ["ts"], as_index = False).apply(_min_gap)

In [1107]:
"""
【步骤二】   <步骤一>数据集  <left join> train_n580
【关联键】   millis_bd_star            millis_n580
【关联条件】 关联后的gpst和GPSTime差尽可能的小，最好是0，如果不是0取差值最小的
"""
dataset = dataset.merge(train_n580, 
                        left_on = "millis_bd_star", 
                        right_on = "millis_n580", 
                        how = 'left').drop_duplicates()

# dataset["gap_2"] = abs(dataset["gpst"] - dataset["GPSTOV"])
dataset["gap_2"] = abs(dataset["gpst"] - dataset["GPSTime"])

def _min_gap2(x):
    df = x.sort_values(by = 'gap_2', ascending = False)
    return df.iloc[-1,:]
    
dataset = dataset.groupby(by = ["ts"], as_index = False).apply(_min_gap2)

In [1108]:
"""
【步骤三】   规整出所有需要的列
"""
# 获取train_bd_star中所有的指标列
train_bd_star_metric_cols = train_bd_star.columns.values.tolist()
# 删除不需要的咧
train_bd_star_metric_cols.remove("ts")
train_bd_star_metric_cols.remove("gpst")
train_bd_star_metric_cols.remove("millis_bd_star")

# 筛选出需要的列
train_data = dataset.drop_duplicates()[
    # bdStar_front.23O
    ["ts", "gpst"] + train_bd_star_metric_cols +
    # stim300_Inertial_data_m.txt
    ["S1", "S2", "S3", "S4", "S5", "S6"] +
    # n580-2023-08-08 05-38-20_.csv
    #[
    #    "Message", "SystemTOV", "GPSTOV", "TimeSinceLastMsg", 
    #    "LinearAccelerationX", "LinearAccelerationY", "LinearAccelerationZ", 
    #    "AngularRateX", "AngularRateY", "AngularRateZ"
    #]
    # refPath_frontAnt
    y_n580_columns
]

print("<train data>: ", train_data.shape)
train_data.head(10)

<train data>:  (82871, 283)


,ts,gpst,C01_B1,C01_B2,C01_B3,C01_B4,C01_B5,C01_B6,C02_B1,C02_B2,...,GPSTime,Latitude,Longitude,H-Ell,VEast,VNorth,VUp,Roll,Pitch,Heading
0,2023-08-08 07:21:15.300000,199293.3000000000,37125346.7030000016,7.0000000000,193321423.8400000036,7.0000000000,-31.1130000000,47.0000000000,NaN,NaN,...,199293.3000000000,31.9395290905,118.7859434828,21.4460000000,0.0010000000,-0.0020000000,0.0020000000,-0.0413317298,-0.3708821932,156.9923996655
1,2023-08-08 07:21:15.400000,199293.4000000000,37125347.2969999984,7.0000000000,193321426.9339999855,7.0000000000,-30.8980000000,47.0000000000,NaN,NaN,...,199293.4000000000,31.9395290902,118.7859434829,21.4460000000,0.0000000000,-0.0010000000,0.0020000000,-0.0410819042,-0.3729177739,156.9925125165
2,2023-08-08 07:21:15.500000,199293.5000000000,37125347.9060000032,7.0000000000,193321430.0740000010,7.0000000000,-31.3590000000,47.0000000000,NaN,NaN,...,199293.5000000000,31.9395290902,118.7859434828,21.4460000000,-0.0010000000,-0.0010000000,0.0030000000,-0.0400638600,-0.3726792199,156.9921140169
3,2023-08-08 07:21:15.600000,199293.6000000000,37125348.5080000013,7.0000000000,193321433.2030000091,7.0000000000,-31.3710000000,47.0000000000,NaN,NaN,...,199293.6000000000,31.9395290895,118.7859434830,21.4460000000,0.0010000000,-0.0040000000,0.0020000000,-0.0399259750,-0.3764609266,156.9923738287
4,2023-08-08 07:21:15.700000,199293.7000000000,37125349.1019999981,7.0000000000,193321436.3199999928,7.0000000000,-31.1450000000,47.0000000000,NaN,NaN,...,199293.7000000000,31.9395290873,118.7859434844,21.4460000000,0.0010000000,-0.0030000000,0.0030000000,-0.0411398152,-0.3906621670,156.9924337479
5,2023-08-08 07:21:15.800000,199293.8000000000,37125349.6950000003,7.0000000000,193321439.4219999909,7.0000000000,-30.9650000000,47.0000000000,NaN,NaN,...,199293.8000000000,31.9395290856,118.7859434850,21.4450000000,0.0010000000,-0.0010000000,0.0030000000,-0.0426254090,-0.4004924096,156.9938166645
6,2023-08-08 07:21:15.900000,199293.9000000000,37125350.2969999984,7.0000000000,193321442.5509999990,7.0000000000,-31.2340000000,47.0000000000,NaN,NaN,...,199293.9000000000,31.9395290839,118.7859434860,21.4450000000,0.0000000000,-0.0010000000,0.0030000000,-0.0461863154,-0.4109220896,156.9954651498
7,2023-08-08 07:21:16.0,199294.0000000000,37125350.8980000019,7.0000000000,193321445.6680000126,7.0000000000,-31.2380000000,47.0000000000,NaN,NaN,...,199294.0000000000,31.9395290832,118.7859434865,21.4450000000,-0.0000000000,-0.0010000000,0.0010000000,-0.0465356321,-0.4156063959,156.9952945479
8,2023-08-08 07:21:16.100000,199294.1000000000,37125351.4919999987,7.0000000000,193321448.7730000019,7.0000000000,-31.1800000000,47.0000000000,NaN,NaN,...,199294.1000000000,31.9395290835,118.7859434864,21.4450000000,-0.0000000000,-0.0010000000,0.0010000000,-0.0465072354,-0.4135588133,156.9951352840
9,2023-08-08 07:21:16.200000,199294.2000000000,37125352.0939999968,7.0000000000,193321451.9020000100,7.0000000000,-31.2810000000,47.0000000000,NaN,NaN,...,199294.2000000000,31.9395290834,118.7859434864,21.4450000000,0.0000000000,-0.0010000000,0.0020000000,-0.0466181748,-0.4145989491,156.9952988620


In [1109]:
"""
【步骤四】   删除结果中输出列缺失的行
【注意】     列Week~Heading 为机器学习中的Y值 如果关联之后这部分有缺失则剔除
"""
gps_time_series = train_data["GPSTime"]
gps_time_na_index = gps_time_series[gps_time_series.isna()].index.tolist()
train_data = train_data.drop(gps_time_na_index)

In [1110]:
"""
【步骤五】   保存文件
"""
train_data_path = "result/train-data.csv"
train_data.to_csv(train_data_path, index = None, sep = ',')

### 卫星核心指标计算

In [1111]:
"""
【步骤一】   读取清洗过的业务数据集
"""
gps_sparse_data_path = "result/train-data.csv"
gps_sparse_data = pd.read_csv(gps_sparse_data_path)
print("<GPS Sparse Table shape>: ", gps_sparse_data.shape)

<GPS Sparse Table shape>:  (81962, 283)


In [1112]:
"""
【步骤二】   获取有效列以及相关列索引用于后续计算
"""
# GPS数据列
gps_columns = gps_sparse_data.columns.tolist()
__gps_columns_size = len(gps_columns)

# GPS数据时间列
__gps_time_columns_start_idx = 0
__gps_time_columns_end_idx = 1
gps_time_columns = gps_data_columns[__gps_time_columns_start_idx: __gps_time_columns_end_idx + 1]

# GPS数据卫星指标组列
__gps_satellite_metric_columns_start_idx = 2
__gps_satellite_metric_columns_end_idx = 265
gps_satellite_metric_columns = gps_columns[__gps_satellite_metric_columns_start_idx: __gps_satellite_metric_columns_end_idx + 1]
__gps_satellite_metric_columns_size = len(gps_satellite_metric_columns)

# GPS数据衍生指标列
gps_derivative_metric_columns = gps_columns[__gps_satellite_metric_columns_end_idx + 1:]
__gps_derivative_metric_columns_size = len(gps_derivative_metric_columns)

In [1113]:
"""
【步骤三】   对卫星核心指标进行行处理
【业务描述】 在一个时间点中，卫星核心指标中小于4颗卫星的核心指标的这个时间点位的数据不保留
【注意】     每个时间点的数据没有关联
"""

__satellite_size_threshold = 4
__metric_size_per_satellite = 6
__satellite_metric_size_threshold = __satellite_size_threshold * __metric_size_per_satellite

# 取出所有时间点卫星核心数据
gps_satellite_metric = gps_sparse_data.loc[:,gps_satellite_metric_columns]
# 查找核心数据中核心指标小于__satellite_metric_size_threshold的数据
satellite_not_null_count_series = gps_satellite_metric.notnull().sum(axis = 1)
# 获取核心指标小于__satellite_metric_size_threshold的数据行索引
drop_rows_index = satellite_not_null_count_series[satellite_not_null_count_series < 24].index
effective_gps_sparse_data = gps_sparse_data.drop(drop_rows_index)

print("<Effectived GPS Sparse Table shape>: ", effective_gps_sparse_data.shape)

<Effectived GPS Sparse Table shape>:  (81448, 283)


##### 卫星核心指标处理逻辑

【业务描述】在一个时间点中，所有卫星的多个核心指标可能是参差不齐的，需要通过列站位候补的方式去除其中缺失值，同时保留预定数量的列。

例如：保留4列

【例子-1】

处理前：

```shell
col:   A    B    C    D    F
value: 1    NA   4    3    7
```

处理后：

```shell
col:   X1   X2   X3   X4
value: 1    4    3    7
```

【例子-2】

处理前：

```shell
col:   A    B    C    D    F
value: 1    3    4    3    7
```


处理后：

```shell
col:   X1   X2   X3   X4
value: 1    3    4    3
```

【例子-3】

处理前：

```shell
col:   A    B    C    D    F    X   Y
value: NA   NA   4    3    7    9   1
```

处理后：

```shell
col:   X1   X2   X3   X4
value: 4    3    7    9
```

In [1114]:
"""
【步骤四】   定义行缺失值补齐压缩处理函数
"""

# 生成变换之后核心指标的列名，以X1, X2...进行编号
gpsm_cols = np.char.add(["X"], np.arange(1, __satellite_metric_size_threshold + 1).astype(str)).tolist()

def __satellite_metric_NA(row): 
    # row: pandas.core.series.Series
    # series -> ndarray
    arr = np.array(row.tolist())
    # drop na value
    notnan_arr = arr[~np.isnan(arr)]
    # get for `__satellite_metric_size_threshold`
    m = notnan_arr[: __satellite_metric_size_threshold]
    return pd.Series(m, index = gpsm_cols)

In [1115]:
"""
【步骤五】   列运算
"""

# 取出所有时间点卫星核心数据
gps_satellite_metric = effective_gps_sparse_data.loc[:,gps_satellite_metric_columns]

# 行运算
# 注意：行运算（axis = 1）
#      列运算（axis = 0）（默认）
effective_satellite_sparse_data = gps_satellite_metric.apply(__satellite_metric_NA, axis = 1)
effective_satellite_sparse_data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24
0,37125346.7030000016,7.0000000000,193321423.8400000036,7.0000000000,-31.1130000000,47.0000000000,36607129.5000000000,5.0000000000,190622928.8630000055,5.0000000000,...,198533386.5390000045,6.0000000000,-30.4340000000,41.0000000000,35888079.7419999987,8.0000000000,186878640.4180000126,8.0000000000,525.6370000000,51.0000000000
1,37125347.2969999984,7.0000000000,193321426.9339999855,7.0000000000,-30.8980000000,47.0000000000,36607129.9689999968,5.0000000000,190622931.2849999964,5.0000000000,...,198533389.5660000145,6.0000000000,-30.1290000000,41.0000000000,35888069.6480000019,8.0000000000,186878587.8479999900,8.0000000000,525.7460000000,51.0000000000
2,37125347.9060000032,7.0000000000,193321430.0740000010,7.0000000000,-31.3590000000,47.0000000000,36607130.4299999997,5.0000000000,190622933.7109999955,5.0000000000,...,198533392.6290000081,6.0000000000,-30.7770000000,41.0000000000,35888059.5549999997,8.0000000000,186878535.3050000072,8.0000000000,525.4180000000,51.0000000000
3,37125348.5080000013,7.0000000000,193321433.2030000091,7.0000000000,-31.3710000000,47.0000000000,36607130.8980000019,5.0000000000,190622936.1599999964,5.0000000000,...,198533395.6910000145,6.0000000000,-30.7340000000,41.0000000000,35888049.4689999968,8.0000000000,186878482.7730000019,8.0000000000,525.2970000000,51.0000000000
4,37125349.1019999981,7.0000000000,193321436.3199999928,7.0000000000,-31.1450000000,47.0000000000,36607131.3830000013,5.0000000000,190622938.6019999981,5.0000000000,...,198533398.7190000117,6.0000000000,-30.3980000000,41.0000000000,35888039.3669999987,8.0000000000,186878430.2150000036,8.0000000000,525.6840000000,51.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81957,37198657.1250000000,7.0000000000,193703167.9410000145,7.0000000000,-51.7420000000,44.0000000000,36149275.4299999997,7.0000000000,188238771.0629999936,7.0000000000,...,186117876.8479999900,5.0000000000,-547.9730000000,34.0000000000,35724095.6639999971,8.0000000000,186024737.0430000126,8.0000000000,-251.9060000000,49.0000000000
81958,37198658.1410000026,7.0000000000,193703173.1879999936,7.0000000000,-52.5120000000,44.0000000000,36149275.5469999984,7.0000000000,188238771.6990000010,7.0000000000,...,186117931.5509999990,5.0000000000,-547.7620000000,34.0000000000,35724100.5080000013,8.0000000000,186024762.2730000019,8.0000000000,-252.6840000000,49.0000000000
81959,37198659.1480000019,7.0000000000,193703178.4410000145,7.0000000000,-52.6950000000,44.0000000000,36149275.6880000010,7.0000000000,188238772.3630000055,7.0000000000,...,186117986.3549999893,5.0000000000,-548.0630000000,34.0000000000,35724105.3519999981,8.0000000000,186024787.5040000081,8.0000000000,-252.3440000000,48.0000000000
81960,37198660.1480000019,7.0000000000,193703183.6480000019,7.0000000000,-52.0120000000,44.0000000000,36149275.7969999984,7.0000000000,188238772.9569999874,7.0000000000,...,186118041.1019999981,5.0000000000,-547.6020000000,35.0000000000,35724110.1880000010,8.0000000000,186024812.7229999900,8.0000000000,-252.0660000000,48.0000000000


In [1116]:
"""
【步骤六】   数据集列拼接（规整有效宽表列）
"""

# 删除处理前的稀疏的卫星核心指标
effective_gps_sparse_data = effective_gps_sparse_data.drop(gps_satellite_metric_columns, axis = 1)

# 将处理后的稠密的卫星核心指标替换到原有数据集的第二列之后
gps_train_data = pd.concat([
    effective_gps_sparse_data[gps_time_columns],
    effective_satellite_sparse_data, 
    effective_gps_sparse_data[gps_derivative_metric_columns]], axis = 1)

In [1117]:
print("<gps_train_data shape>: ", gps_train_data.shape)
gps_train_data.head(5)

<gps_train_data shape>:  (81448, 43)


,ts,gpst,X1,X2,X3,X4,X5,X6,X7,X8,...,GPSTime,Latitude,Longitude,H-Ell,VEast,VNorth,VUp,Roll,Pitch,Heading
0,2023-08-08 07:21:15.300000,199293.3000000000,37125346.7030000016,7.0000000000,193321423.8400000036,7.0000000000,-31.1130000000,47.0000000000,36607129.5000000000,5.0000000000,...,199293.3000000000,31.9395290905,118.7859434828,21.4460000000,0.0010000000,-0.0020000000,0.0020000000,-0.0413317298,-0.3708821932,156.9923996655
1,2023-08-08 07:21:15.400000,199293.4000000000,37125347.2969999984,7.0000000000,193321426.9339999855,7.0000000000,-30.8980000000,47.0000000000,36607129.9689999968,5.0000000000,...,199293.4000000000,31.9395290902,118.7859434829,21.4460000000,0.0000000000,-0.0010000000,0.0020000000,-0.0410819042,-0.3729177739,156.9925125165
2,2023-08-08 07:21:15.500000,199293.5000000000,37125347.9060000032,7.0000000000,193321430.0740000010,7.0000000000,-31.3590000000,47.0000000000,36607130.4299999997,5.0000000000,...,199293.5000000000,31.9395290902,118.7859434828,21.4460000000,-0.0010000000,-0.0010000000,0.0030000000,-0.0400638600,-0.3726792199,156.9921140169
3,2023-08-08 07:21:15.600000,199293.6000000000,37125348.5080000013,7.0000000000,193321433.2030000091,7.0000000000,-31.3710000000,47.0000000000,36607130.8980000019,5.0000000000,...,199293.6000000000,31.9395290895,118.7859434830,21.4460000000,0.0010000000,-0.0040000000,0.0020000000,-0.0399259750,-0.3764609266,156.9923738287
4,2023-08-08 07:21:15.700000,199293.7000000000,37125349.1019999981,7.0000000000,193321436.3199999928,7.0000000000,-31.1450000000,47.0000000000,36607131.3830000013,5.0000000000,...,199293.7000000000,31.9395290873,118.7859434844,21.4460000000,0.0010000000,-0.0030000000,0.0030000000,-0.0411398152,-0.3906621670,156.9924337479


In [1118]:
"""
【步骤7】   保存计算结果
"""
gps_train_data_path = "result/gps-train-data.csv"
gps_train_data.to_csv(gps_train_data_path, index = None, sep = ',')

### 数据集验证

对预测后的数据集中真实经纬度和预测经纬度同时转换为单位为米的度量，考察模型预测之后具体误差情况

#### 辅助函数

In [5]:
"""
经纬度与地心坐标之间的转换
【参考】https://juejin.cn/s/python%20%E7%BB%8F%E7%BA%AC%E5%BA%A6%E8%BD%AC%E6%8D%A2%E4%B8%BA%E7%9B%B4%E8%A7%92%E5%9D%90%E6%A0%87
"""
from math import sin, cos, radians

# 经纬度 => 地心空间坐标
def lon_lat_to_xyz(lon, lat, R = 6371.0088):
    '''
    Convert lon, lat in degrees to x, y, z coordinates.
    lon: longitude in degrees
    lat: latitude in degrees
    R: radius of the earth
    return: x, y, z
    '''
    lon, lat = radians(lon), radians(lat)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return x, y, z

# 地心空间坐标 => 经纬度
def xyz_to_lon_lat(x, y, z):
    '''
    Convert x, y, z coordinates to lon, lat in degrees.
    x: x coordinate
    y: y coordinate
    z: z coordinate
    return: lon, lat in degrees
    '''
    lon = np.arctan2(y, x)
    lat = np.arctan2(z, np.sqrt(x**2 + y**2))
    return np.degrees(lon), np.degrees(lat)

# 经纬度 => 地心空间坐标
# 参考二：hhave ttps://cloud.tencent.com/developer/ask/sof/78178
def get_cartesian(lat=None,lon=None):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    R = 6371 # radius of the earth
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R *np.sin(lat)
    return x,y,z

"""
[TEST]
"""
print(lon_lat_to_xyz(31.9395290905, 118.7859434828))
print(xyz_to_lon_lat(-2603.4302100655705, -1622.9856887050905, 5583.71039052473))

print(get_cartesian(31.9395290905, 118.7859434828))

(-2603.4302100655705, -1622.9856887050905, 5583.71039052473)
(-148.06047090950003, 61.21405651720003)
(-2603.426614059574, 4738.368671743021, 3370.4114282755722)


In [2]:
"""
计算经纬度之间的距离
"""

"""
pip install geopy
"""
from geopy.distance import geodesic

# p1: (latitude, longitude)
# p2: (latitude, longitude)
def __distance_geopy(p1: tuple, p2: tuple, measure = "m"):
    result = geodesic(p1, p2)
    if measure == "m":
        return result.m
    elif measure == "km":
        return result.km
    else:
        raise Exception('Not support measure: {}!'.format(measure))

"""
pip install haversine
"""
from haversine import haversine, Unit

# p1: (latitude, longitude)
# p2: (latitude, longitude)
def __distance_haversine(p1: tuple, p2: tuple, measure = "m"):
    if measure == "m":
        return haversine(p1, p2, unit = Unit.METERS)
    elif measure == "km":
        return haversine(p1, p2, unit = Unit.KILOMETERS)
    else:
        raise Exception('Not support measure: {}!'.format(measure))

print(__distance_geopy((30.28708,120.12802999999997), (28.7427,115.86572000000001), "m"))
print(__distance_geopy((30.28708,120.12802999999997), (28.7427,115.86572000000001), "km"))

print(__distance_haversine((30.28708,120.12802999999997), (28.7427,115.86572000000001), "m"))
print(__distance_haversine((30.28708,120.12802999999997), (28.7427,115.86572000000001), "km"))

447249.7993541996
447.24979935419964
446721.3440465478
446.7213440465478


In [6]:
# 读取验证集数据

# 真实经度和预测经度
latitude_dataset = pd.read_csv("predict/latitude-predict.csv")[["gpst", "Latitude", "ypred"]].rename(columns = {
    "Latitude" : "latitude",
    "ypred":     "latitude_predict"
})

# 真实纬度和预测纬度
longitude_dataset = pd.read_csv("predict/longitude-predict.csv")[["gpst", "Longitude", "ypred"]].rename(columns = {
    "Longitude" : "longitude",
    "ypred":     "longitude_predict"
})

print("<Latitude Predict Dataset>: ", latitude_dataset.shape)
print("<Longitude Predict Dataset>: ", longitude_dataset.shape)

<Latitude Predict Dataset>:  (16290, 3)
<Longitude Predict Dataset>:  (16290, 3)


In [7]:
# 整合经纬度的实际值和预测值
location_dataset = latitude_dataset.merge(longitude_dataset, left_on = "gpst", right_on = "gpst", how = 'inner', sort = True)
print("<Location Predict Dataset>: ", location_dataset.shape)

<Location Predict Dataset>:  (3214, 5)


In [9]:
# 定义解析经纬度函数
def __transfer_location(row): 
    _gpst = row["gpst"]
    _lat = row["latitude"]
    _lon = row["longitude"]
    _lat_head = row["latitude_predict"]
    _lon_head = row["longitude_predict"]
    x, y, z = lon_lat_to_xyz(_lon, _lat)
    x_head, y_head, z_head = lon_lat_to_xyz(_lon_head, _lat_head)
    # 计算地心坐标各分量的差值
    x_delta, y_delta, z_delta = abs(x - x_head), abs(y - y_head), (z - z_head)
    # 计算真实经纬度和预测经纬度之间的差值
    distance_2d = __distance_geopy((_lat, _lon), (_lat_head, _lon_head), "m")
    return pd.Series([
        _gpst,
        _lat, _lon,
        _lat_head, _lon_head,
        x, y, z,
        x_head, y_head, z_head,
        x_delta, y_delta, z_delta,
        distance_2d
    ], index = [
        "gpst",
        "latitude", "longitude",
        "latitude_predict", "longitude_predict",
        "x", "y", "z",
        "x_head", "y_head", "z_head",
        "x_delta", "y_delta", "z_delta",
        "distance_2d"
    ])

location_lose_result = location_dataset.apply(__transfer_location, axis = 1)
location_lose_result.head(5)

,gpst,latitude,longitude,latitude_predict,longitude_predict,x,y,z,x_head,y_head,z_head,x_delta,y_delta,z_delta,distance_2d
0,199293.8,31.939529,118.785943,31.939523,118.785958,-2603.430210,4738.375217,3370.416083,-2603.431546,4738.37488,3370.415525,0.001336,0.000337,0.000558,1.488114
1,199299.3,31.939529,118.785943,31.939523,118.785958,-2603.430211,4738.375217,3370.416083,-2603.431546,4738.37488,3370.415525,0.001336,0.000337,0.000558,1.487827
2,199301.3,31.939529,118.785943,31.939523,118.785958,-2603.430211,4738.375217,3370.416083,-2603.431546,4738.37488,3370.415525,0.001336,0.000337,0.000558,1.487806
3,199302.2,31.939529,118.785943,31.939523,118.785958,-2603.430210,4738.375217,3370.416084,-2603.431546,4738.37488,3370.415525,0.001336,0.000337,0.000558,1.488316
4,199305.1,31.939529,118.785943,31.939523,118.785958,-2603.430210,4738.375217,3370.416084,-2603.431546,4738.37488,3370.415525,0.001336,0.000337,0.000558,1.488423


In [17]:
location_lose_result.describe()
location_lose_result.describe()[["x_delta", "y_delta", "z_delta", "distance_2d"]]

,x_delta,y_delta,z_delta,distance_2d
count,3214.000000,3214.000000,3214.000000,3214.000000
mean,0.006107,0.004869,0.000110,11.441450
std,0.008088,0.006302,0.011089,12.559118
min,0.000001,0.000001,-0.055557,0.146669
25%,0.001169,0.000645,-0.002480,2.189610
50%,0.003342,0.002251,0.000288,6.485453
75%,0.008129,0.006817,0.002816,16.783675
max,0.176829,0.093742,0.055148,200.608794


In [19]:
d = location_lose_result[location_lose_result["distance_2d"] < 96]
d.describe()[["x_delta", "y_delta", "z_delta", "distance_2d"]]

,x_delta,y_delta,z_delta,distance_2d
count,3212.000000,3212.000000,3212.000000,3212.000000
mean,0.006031,0.004827,0.000108,11.356107
std,0.007391,0.006051,0.011091,12.017926
min,0.000001,0.000001,-0.055557,0.146669
25%,0.001168,0.000645,-0.002479,2.183535
50%,0.003337,0.002249,0.000288,6.457596
75%,0.008118,0.006794,0.002816,16.761050
max,0.069125,0.047306,0.055148,82.139836


In [24]:
x = pd.cut(d["distance_2d"], bins = [0, 1, 5, 10, 20, 50, 100])
pd.value_counts(x)

/var/folders/b9/dy658dr90gn713c0gcy1sbgc0000gn/T/ipykernel_32379/3279606535.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(x)


distance_2d
(1, 5]       1127
(10, 20]      648
(20, 50]      600
(5, 10]       537
(0, 1]        259
(50, 100]      41
Name: count, dtype: int64

In [1240]:
location_loss_result_path = "result/gps-train-data-loss.csv"
location_lose_result.to_csv(location_loss_result_path, index = None, sep = ',')